In [1]:
import pandas as pd
import pypsa
from scripts.__helpers import replace_su, average_every_nhours
import numpy as np
import os
import math

In [2]:
from pypsa.descriptors import get_switchable_as_dense as as_dense

In [3]:
def solve_rolling_horizon(m, o, time_aggregation, horizon):
    
    o.storage_units.cyclic_state_of_charge=False
    o.stores.e_cyclic=False

    o.stores_t.e_min_pu = (
        (m.stores_t.e/m.stores.e_nom)
        .shift(1)
        .resample(time_aggregation).asfreq()
        .shift(-1)
        .fillna(0)
    ).clip(upper=.999).reindex(o.snapshots).fillna(0)

    o.stores.e_initial = m.stores_t.e.iloc[-1]
    
    for i in range(len(o.snapshots)//horizon):

        start= i* horizon
        end = min((i+1)*horizon, len(o.snapshots))
        
        print("optimizing time period between " + str(o.snapshots[start]) + " and " + str(o.snapshots[end-1]))
        
        snapshots = o.snapshots[start:end]
              
        o.optimize(solver_name=solver_name,
                   snapshots = snapshots,
                   assign_all_duals=True,
                   extra_functionality=balancing_market,
                   linearized_unit_commitment = True
              )
        
        
        o.stores.e_initial = o.stores_t.e.loc[snapshots].iloc[-1]

In [4]:
def set_storage_bounds():

    n.storage_units.cyclic_state_of_charge=False
    n.stores.e_cyclic=False

    n.stores_t.e_min_pu = (
        (m.stores_t.e/m.stores.e_nom)
        .shift(1)
        .resample("h").asfreq()
        .shift(-1)
        .fillna(0)
    ).clip(upper=.999).reindex(n.snapshots).fillna(0)

    n.stores.e_initial = m.stores_t.e.iloc[-1]

In [5]:
def get_ordc_integral(excess_reserve, mean, std):
    
    return (
        -0.5*excess_reserve 
        + 0.5*(
            (excess_reserve - mean)*math.erf((excess_reserve-mean)/(std*math.sqrt(2)))
            + math.sqrt(2)*std*math.exp(-(excess_reserve-mean)**2/(2*std**2))/math.sqrt(math.pi)
        )
    )

In [6]:
def determine_cutoff(zone):
    cost_function = pd.Series([get_ordc_integral(i, **ordc_parameters.loc[zone]) for i in range(0,40000,10)], 
                              index = range(0,40000,10))

    return cost_function.diff()[cost_function.diff()/pd.Series(cost_function.index).diff().median()<=-0.00001].index[-1]

In [7]:
def get_break_points(n_ints, zone, cut_off):
    
    break_points = pd.DataFrame(index=range(n_ints+1))
    break_points["cap_break_points"] = list(np.arange(0, cut_off[zone], cut_off[zone]/n_ints)) + [2*cut_off[zone]]
    break_points["welfare_break_points"] = [get_ordc_integral(i, **ordc_parameters.loc[zone]) for i in break_points.cap_break_points]
    break_points.set_index(pd.Series(zone,index=break_points.index), inplace=True,append=True)
    break_points = break_points.reorder_levels([1,0])
    
    return break_points

In [8]:
def get_linear_function(p, q):
    m = (p[1] - q[1])/(p[0] - q[0])
    b = (p[0]*q[1] - p[1]*q[0])/(p[0] - q[0])
    return m, b

In [9]:
def get_line_parameters(ordc_parameters, n_ints):
    cut_off = pd.Series(index=ordc_parameters.index, data=[determine_cutoff(i) for i in ordc_parameters.index])

    break_points = pd.DataFrame()
    for zone in ordc_parameters.index:

        break_points = pd.concat([break_points, get_break_points(n_ints, zone, cut_off)])

    line_parameters = pd.DataFrame()
    for zone in ordc_parameters.index:
        j = 0    
        for i in break_points.loc[zone].index[:-1]:
            m, b = get_linear_function(break_points.loc[zone, i], break_points.loc[zone, i+1])
            line_parameters = pd.concat([line_parameters, pd.Series(index=["a", "b"], data=[m, b], name=(zone, j))],axis=1)
            j+=1

    return line_parameters.T.set_index((i for i in line_parameters.columns))

Index(['AL00', 'AT00', 'BA00', 'BE00', 'BEOF', 'BG00', 'CH00', 'CY00', 'CZ00',
       'DE00', 'DEKF', 'DKBH', 'DKE1', 'DKKF', 'DKNS', 'DKW1', 'EE00', 'ES00',
       'FI00', 'FR00', 'GR00', 'GR03', 'HR00', 'HU00', 'IE00', 'ITCA', 'ITCN',
       'ITCS', 'ITN1', 'ITS1', 'ITSA', 'ITSI', 'LT00', 'LUB1', 'LUF1', 'LUG1',
       'LUV1', 'LV00', 'ME00', 'MK00', 'MT00', 'NL00', 'NLLL', 'NOM1', 'NON1',
       'NOS0', 'PL00', 'PT00', 'RO00', 'RS00', 'SE01', 'SE02', 'SE03', 'SE04',
       'SI00', 'SK00', 'UK00', 'UKNI'],
      dtype='object', name='Bus')

In [41]:
def balancing_market(n, snapshots):
    
    m = n.model

    reserve_gens = n.generators.loc[n.generators.bus.map(n.buses.carrier) == "electricity"]
    reserve_gens = reserve_gens.query("carrier in @reserve_participation_generators")

    coord_reserve_gens = reserve_gens.index
    coord_reserve_gens.name = "Generator-fix"

    gen_r = m.add_variables(lower=0, name="Generator-r", coords = [snapshots, coord_reserve_gens])

    lhs = m.constraints["Generator-fix-p-upper"].lhs
    rhs = m.constraints["Generator-fix-p-upper"].rhs
    m.remove_constraints("Generator-fix-p-upper")
    m.add_constraints(lhs + gen_r <= rhs, name= "Generator-fix-p-upper")

    committable = reserve_gens.query("committable == True").copy()

    committable_grouper = pd.Series(committable.index, committable.index)
    committable_grouper.name="Generator-com"

    lhs = m.constraints["Generator-com-p-upper"].lhs
    rhs = m.constraints["Generator-com-p-upper"].rhs

    lhs += gen_r.loc[:, committable.index].groupby(committable_grouper).sum()
    m.remove_constraints("Generator-com-p-upper")
    m.add_constraints( lhs <= rhs, name="Generator-com-p-upper")

    gen_status = m["Generator-status"]

    reserve_gen_status = gen_status.loc[:, committable.index]

    m.add_constraints(
        gen_r.loc[:, committable.index].groupby(committable_grouper).sum()
        <= (
            reserve_gen_status*
            (
                committable.groupby(committable_grouper).ramp_limit_up.sum()
                .multiply(delivery_time_reserves)
                .multiply(committable.groupby(committable_grouper).p_nom.sum())     
            )
        ),
        name = "Generator-com-ramp_upper"
    )

    res_shedding = m.add_variables(
        lower=0, 
        name="Bus-reserve_shedding", 
        coords=[snapshots, reserve_requirements.index]
    )

    storage = n.links.loc[n.links.type == "discharging"]

    storage_link_r = m.add_variables(lower = 0, name="Link-r", coords = m["Link-p"].loc[:, storage.index].coords)

    link_p = m["Link-p"]

    m.add_constraints(
        link_p.loc[:, storage.index] + storage_link_r <= n.links.p_nom.to_xarray(),
        name = "Link-capacity_upper"
    );

    store_e = m["Store-e"]

    store_link_map = n.stores.set_index("bus",append=True).reset_index("Store").reindex(storage.bus0)["Store"]
    store_link_map.index = storage.index
    store_link_map.index.name = "Link"
    store_link_map.name="Store"

    m.add_constraints(
        storage_link_r.groupby(store_link_map.to_xarray()).sum() <= store_e,
        name="Link-reserve_reservoir_constraint"
    )

    gen_grouper = reserve_gens.bus
    gen_grouper.name = "Bus"

    storage_grouper = storage.bus1
    storage_grouper.name = "Bus"

    if balancing_market_design == "ORDC":

        excess_r = m.add_variables(
            lower = 0,
            upper= 40,
            name = "Bus-excess_r",
            coords = [snapshots, reserve_requirements.index]
        )

        m.add_constraints(
            gen_r.groupby(gen_grouper.to_xarray()).sum() 
            + storage_link_r.groupby(storage_grouper.to_xarray()).sum()
            + res_shedding
            - excess_r
            >= 0,
            name="Bus-reserve_balance"
        )

        
        a = linear_ordc_approximation["a"]
        b = linear_ordc_approximation["b"]

        a.index.names = ["Bus", "break_point"]
        b.index.names = ["Bus", "break_point"]


        ordc_cost_term = m.add_variables(coords=excess_r.coords, name="Bus-ORDC_cost")
        
        m.add_constraints(
            ordc_cost_term >= (a.to_xarray()*excess_r + b.to_xarray())*VOLL,
            name="Bus-ORDC"
        )

        obj = m.objective
        m.add_objective(obj + res_shedding.sum()*1e5 + ordc_cost_term.sum(), overwrite=True) 

    else:

        m.add_constraints(
            gen_r.groupby(gen_grouper.to_xarray()).sum() 
            + storage_link_r.groupby(storage_grouper.to_xarray()).sum()
            + res_shedding
            >= reserve_requirements.reindex(n.buses.query("carrier == 'electricity'").index).dropna().to_xarray(),
            name="Bus-reserve_balance"
        )

        obj = m.objective

        m.add_objective(obj + res_shedding.sum()*1e5, overwrite=True)

In [11]:
target_year = 2029

In [12]:
solved_network = "results/networks/base/cy1992_ty2028.nc"

In [42]:
n = pypsa.Network("resources/networks/base/cy1992_ty2028.nc")

INFO:pypsa.io:Imported network cy1992_ty2028.nc has buses, generators, links, loads, storage_units


In [14]:
years = range(2025, 2034)

In [68]:
reserve_requirements = pd.read_excel("data/pemmdb.xlsx", "Reserve Requirements",index_col=[0,1])[["FCR (MW)", "FRR (MW)"]].sum(axis=1)#.loc[:, target_year]
reserve_requirements = reserve_requirements.unstack(1).reindex(years, axis=1).interpolate(axis=1)[target_year]
reserve_requirements.index.name = "Bus"
reserve_requirements = reserve_requirements[reserve_requirements>0]

In [44]:
solver_name = "highs"

In [45]:
VOLL = 10e3

In [46]:
reserve_participation_generators = ['CCGT', 'OCGT', 'oil', 'biomass', 'other', 'lignite', 'nuclear', 'coal', "onwind"]

In [47]:
balancing_market_design = "ORDC"

In [48]:
for su in n.storage_units.index:
    replace_su(n, su);

In [49]:
storage_preopt_aggregation = "24h"

In [50]:
delivery_time_reserves = 15 # in minutes
delivery_time_reserves = delivery_time_reserves/60

In [51]:
n.madd("Generator", 
       n.buses.query("carrier == 'electricity'").index + " load-shedding", 
       bus = n.buses.query("carrier == 'electricity'").index, 
       p_nom = 1e6, 
       marginal_cost = VOLL
      )

Index(['AL00 load-shedding', 'AT00 load-shedding', 'BA00 load-shedding',
       'BE00 load-shedding', 'BG00 load-shedding', 'CH00 load-shedding',
       'CY00 load-shedding', 'CZ00 load-shedding', 'DE00 load-shedding',
       'DEKF load-shedding', 'DKE1 load-shedding', 'DKKF load-shedding',
       'DKW1 load-shedding', 'EE00 load-shedding', 'ES00 load-shedding',
       'FI00 load-shedding', 'FR00 load-shedding', 'GR00 load-shedding',
       'GR03 load-shedding', 'HR00 load-shedding', 'HU00 load-shedding',
       'IE00 load-shedding', 'ITCA load-shedding', 'ITCN load-shedding',
       'ITCS load-shedding', 'ITN1 load-shedding', 'ITS1 load-shedding',
       'ITSA load-shedding', 'ITSI load-shedding', 'ITVI load-shedding',
       'LT00 load-shedding', 'LUG1 load-shedding', 'LV00 load-shedding',
       'ME00 load-shedding', 'MK00 load-shedding', 'MT00 load-shedding',
       'NL00 load-shedding', 'NOM1 load-shedding', 'NON1 load-shedding',
       'NOS0 load-shedding', 'PL00 load-shedding', 

In [52]:
ordc_parameters = pd.read_hdf("data/ordc_parameters.h5", "ordc_parameters")

In [25]:
m = average_every_nhours(n, storage_preopt_aggregation)

In [26]:
m.generators["p_min_pu"] = 0

In [27]:
m.stores.e_cyclic = True

In [28]:
m.optimize(
    solver_name="highs",
)

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  6.66it/s]
INFO:linopy.io: Writing time: 4.19s
INFO:linopy.solvers:Log file at /tmp/highs.log.


Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d-dirty]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
83950 rows, 412780 cols, 613150 nonzeros
83950 rows, 307856 cols, 442716 nonzeros
83949 rows, 307812 cols, 442666 nonzeros
Presolve : Reductions: rows 83949(-937679); columns 307812(-125443); elements 442666(-1269535)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -8.0836438673e+02 Pr: 45663(4.28785e+08); Du: 0(8.59527e-09) 1s
       6869     5.4192674366e+09 Pr: 43083(2.89127e+08); Du: 0(3.55448e-06) 6s
      14168     8.0570654652e+09 Pr: 41207(2.38532e+08); Du: 0(4.35911e-06) 12s
      21193     1.0173276823e+10 Pr: 40103(2.02545e+08); Du: 0(7.73571e-06) 17s
      28725     1.3070855464e+10 Pr: 37506(1.92302e+08); Du: 0(9.78521e-06) 22s
      37244     1.6488953248e+10 Pr: 35388(1.90992e+08); Du: 0(1.08806e-05) 27s
      46892     2.1340637681e+10 Pr: 32033(1.09302

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 433255 primals, 1021628 duals
Objective: 3.70e+10
Solver model: available
Solver message: optimal

/trinity/home/thomgeo/.conda/envs/pypsa-training/lib/python3.10/site-packages/linopy/common.py:124: UserWarning: Coordinates across variables not equal. Perform outer join.
  warn("Coordinates across variables not equal. Perform outer join.", UserWarning)
INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-fix-p-ramp_limit_up, Generator-fix-p-ramp_limit_down, Link-fix-p-lower, Link-fix-p-upper, Store-fix-e-lower, Store-fix-e-upper, Store-energy_balance were not assigned to the network.


('ok', 'optimal')

In [56]:
dispatchable = ['CCGT', 'OCGT', 'oil', 'biomass', 'other', 'lignite', 'nuclear', 'coal']

n.generators.loc[n.generators.query("carrier in @dispatchable").index, "committable"] = True

n.generators.shut_down_cost = n.generators.start_up_cost

In [57]:
linear_ordc_approximation = get_line_parameters(ordc_parameters, n_ints = 5)

In [81]:
m = o.copy()

In [82]:
solve_rolling_horizon(m, n, "h", 24)

optimizing time period between 2010-01-01 00:00:00 and 2010-01-01 23:00:00


INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 140.41it/s]
INFO:linopy.io: Writing time: 0.65s
INFO:linopy.solvers:Log file at /tmp/highs.log.


Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d-dirty]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
43253 rows, 41174 cols, 150098 nonzeros
34528 rows, 37655 cols, 126673 nonzeros
Presolve : Reductions: rows 34528(-67096); columns 37655(-9601); elements 126673(-97170)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.1432000000e+07 Ph1: 6424(1.85644e+07); Du: 696(11432) 0s
      16190     2.7734457518e+08 Pr: 0(0); Du: 0(1.42109e-14) 0s
      16190     2.7734457518e+08 Pr: 0(0); Du: 0(1.42109e-14) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 16190
Objective value     :  2.7734457518e+08
HiGHS run time      :          0.66


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 47256 primals, 101624 duals
Objective: 2.77e+08
Solver model: available
Solver message: optimal

/trinity/home/thomgeo/.conda/envs/pypsa-training/lib/python3.10/site-packages/linopy/common.py:124: UserWarning: Coordinates across variables not equal. Perform outer join.
  warn("Coordinates across variables not equal. Perform outer join.", UserWarning)
/trinity/home/thomgeo/.conda/envs/pypsa-training/lib/python3.10/site-packages/linopy/common.py:124: UserWarning: Coordinates across variables not equal. Perform outer join.
  warn("Coordinates across variables not equal. Perform outer join.", UserWarning)
INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Bus-ORDC were not assigned to the network.


optimizing time period between 2010-01-02 00:00:00 and 2010-01-02 23:00:00


...

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 142.48it/s]
INFO:linopy.io: Writing time: 0.64s
INFO:linopy.solvers:Log file at /tmp/highs.log.


Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d-dirty]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
43197 rows, 41193 cols, 150005 nonzeros
34519 rows, 37660 cols, 126655 nonzeros
Presolve : Reductions: rows 34519(-67477); columns 37660(-9596); elements 126655(-97814)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.1400000000e+07 Ph1: 6425(1.85639e+07); Du: 696(11400) 0s
      16639     3.1543485505e+08 Pr: 0(0); Du: 0(8.52651e-14) 0s
      16639     3.1543485505e+08 Pr: 0(0); Du: 0(8.52651e-14) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 16639
Objective value     :  3.1543485505e+08
HiGHS run time      :          0.68


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 47256 primals, 101996 duals
Objective: 3.15e+08
Solver model: available
Solver message: optimal

/trinity/home/thomgeo/.conda/envs/pypsa-training/lib/python3.10/site-packages/linopy/common.py:124: UserWarning: Coordinates across variables not equal. Perform outer join.
  warn("Coordinates across variables not equal. Perform outer join.", UserWarning)
/trinity/home/thomgeo/.conda/envs/pypsa-training/lib/python3.10/site-packages/linopy/common.py:124: UserWarning: Coordinates across variables not equal. Perform outer join.
  warn("Coordinates across variables not equal. Perform outer join.", UserWarning)
INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Bus-ORDC were not assigned to the network.


optimizing time period between 2010-01-04 00:00:00 and 2010-01-04 23:00:00


KeyboardInterrupt: 

In [ ]:
m = n.optimize.create_model(
    linearized_unit_commitment=True,
    snapshots = snapshots,
)

In [ ]:
balancing_market = "ORDC"

In [ ]:
n.generators.min_down_time = 0
n.generators.min_down_time = 0

In [ ]:
n.optimize(
    linearized_unit_commitment=True,
    snapshots = n.snapshots[:24],
    solver_name="highs",
    extra_functionality=reserve_model,
    assign_all_duals=True
)

In [62]:
snapshots = n.snapshots[:24]

In [69]:
m = n.optimize.create_model(snapshots = snapshots, linearized_unit_commitment=True)

reserve_gens = n.generators.loc[n.generators.bus.map(n.buses.carrier) == "electricity"]
reserve_gens = reserve_gens.query("carrier in @reserve_participation_generators")

coord_reserve_gens = reserve_gens.index
coord_reserve_gens.name = "Generator-fix"

gen_r = m.add_variables(lower=0, name="Generator-r", coords = [snapshots, coord_reserve_gens])

lhs = m.constraints["Generator-fix-p-upper"].lhs
rhs = m.constraints["Generator-fix-p-upper"].rhs
m.remove_constraints("Generator-fix-p-upper")
m.add_constraints(lhs + gen_r <= rhs, name= "Generator-fix-p-upper")

committable = reserve_gens.query("committable == True").copy()

committable_grouper = pd.Series(committable.index, committable.index)
committable_grouper.name="Generator-com"

lhs = m.constraints["Generator-com-p-upper"].lhs
rhs = m.constraints["Generator-com-p-upper"].rhs

lhs += gen_r.loc[:, committable.index].groupby(committable_grouper).sum()
m.remove_constraints("Generator-com-p-upper")
m.add_constraints( lhs <= rhs, name="Generator-com-p-upper")

gen_status = m["Generator-status"]

reserve_gen_status = gen_status.loc[:, committable.index]

m.add_constraints(
    gen_r.loc[:, committable.index].groupby(committable_grouper).sum()
    <= (
        reserve_gen_status*
        (
            committable.groupby(committable_grouper).ramp_limit_up.sum()
            .multiply(delivery_time_reserves)
            .multiply(committable.groupby(committable_grouper).p_nom.sum())     
        )
    ),
    name = "Generator-com-ramp_upper"
)

Constraint `Generator-com-ramp_upper` (snapshot: 24, Generator-com: 127):
-------------------------------------------------------------------------
[2010-01-01 00:00:00, AT00 biomass]: +1 Generator-r[2010-01-01 00:00:00, AT00 biomass] - 1580 Generator-status[2010-01-01 00:00:00, AT00 biomass]  ≤ 0
[2010-01-01 00:00:00, AT00 oil]: +1 Generator-r[2010-01-01 00:00:00, AT00 oil] - 144 Generator-status[2010-01-01 00:00:00, AT00 oil]               ≤ 0
[2010-01-01 00:00:00, AT00 other]: +1 Generator-r[2010-01-01 00:00:00, AT00 other] - 1902 Generator-status[2010-01-01 00:00:00, AT00 other]        ≤ 0
[2010-01-01 00:00:00, BA00 lignite]: +1 Generator-r[2010-01-01 00:00:00, BA00 lignite] - 524.4 Generator-status[2010-01-01 00:00:00, BA00 lignite] ≤ 0
[2010-01-01 00:00:00, BE00 biomass]: +1 Generator-r[2010-01-01 00:00:00, BE00 biomass] - 2180 Generator-status[2010-01-01 00:00:00, BE00 biomass]  ≤ 0
[2010-01-01 00:00:00, BE00 nuclear]: +1 Generator-r[2010-01-01 00:00:00, BE00 nuclear] - 1558 Gen

In [70]:
res_shedding = m.add_variables(
    lower=0, 
    name="Bus-reserve_shedding", 
    coords=[snapshots, reserve_requirements.index]
)

In [74]:
storage = n.links.loc[n.links.type == "discharging"]

storage_link_r = m.add_variables(lower = 0, name="Link-r", coords = m["Link-p"].loc[:, storage.index].coords)

link_p = m["Link-p"]

m.add_constraints(
    link_p.loc[:, storage.index] + storage_link_r <= n.links.p_nom.to_xarray(),
    name = "Link-capacity_upper"
);

store_e = m["Store-e"]

store_link_map = n.stores.set_index("bus",append=True).reset_index("Store").reindex(storage.bus0)["Store"]

store_link_map.index = storage.index

store_link_map.index.name = "Link"

store_link_map.name="Store"

In [76]:
m.add_constraints(
    storage_link_r.groupby(store_link_map.to_xarray()).sum() <= store_e,
    name="Link-reserve_reservoir_constraint"
)

gen_grouper = reserve_gens.bus
gen_grouper.name = "Bus"

storage_grouper = storage.bus1
storage_grouper.name = "Bus"

In [77]:
excess_r = m.add_variables(
        lower = 0,
        name = "Bus-excess_r",
        coords = [snapshots, reserve_requirements.index]
    )

In [78]:
m.add_constraints(
    gen_r.groupby(gen_grouper.to_xarray()).sum() 
    + storage_link_r.groupby(storage_grouper.to_xarray()).sum()
    + res_shedding
    >= excess_r,
    name="Bus-reserve_balance"
)

Constraint `Bus-reserve_balance` (Bus: 49, snapshot: 24):
---------------------------------------------------------
[AL00, 2010-01-01 00:00:00]: +1 Generator-r[2010-01-01 00:00:00, AL00 onwind] + 1 Link-r[2010-01-01 00:00:00, AL00 reservoir discharger] + 1 Bus-reserve_shedding[2010-01-01 00:00:00, AL00] - 1 Bus-excess_r[2010-01-01 00:00:00, AL00]                                                                                                   ≥ -0.0
[AL00, 2010-01-01 01:00:00]: +1 Generator-r[2010-01-01 01:00:00, AL00 onwind] + 1 Link-r[2010-01-01 01:00:00, AL00 reservoir discharger] + 1 Bus-reserve_shedding[2010-01-01 01:00:00, AL00] - 1 Bus-excess_r[2010-01-01 01:00:00, AL00]                                                                                                   ≥ -0.0
[AL00, 2010-01-01 02:00:00]: +1 Generator-r[2010-01-01 02:00:00, AL00 onwind] + 1 Link-r[2010-01-01 02:00:00, AL00 reservoir discharger] + 1 Bus-reserve_shedding[2010-01-01 02:00:00, AL00] - 1 Bus-excess_r[20

In [ ]:
a = linear_ordc_approximation["a"]
b = linear_ordc_approximation["b"]

a.index.names = ["Bus", "break_point"]
b.index.names = ["Bus", "break_point"]


ordc_cost_term = m.add_variables(coords=excess_r.coords, name="Bus-ORDC_cost")

m.add_constraints(
    ordc_cost_term >= (a.to_xarray()*excess_r + b.to_xarray())*VOLL,
    name="Bus-ORDC"
)

obj = m.objective
m.add_objective(obj + res_shedding.sum()*1e5 + ordc_cost_term.sum(), overwrite=True) 

In [ ]:
m.solve(solver_name="highs")

In [ ]:
m.solution["Link-r"].to_dataframe()

In [ ]:
m.solution["Store-e"].to_dataframe()

In [ ]:
m.solution["Generator-r"].to_dataframe()

In [ ]:
def duc(ts):
    return ts.sort_values(ascending=False).reset_index(drop=True)

In [ ]:
for col in n.buses_t.mu_reserve_balance.columns:
    duc(n.buses_t.mu_reserve_balance[col]).plot()
    

In [ ]:
n.optimize(
    solver_name="highs",
    assign_all_duals=True,
    extra_functionality=reserves_unit_commitment,
    linearized_unit_commitment=True,
    snapshots=n.snapshots[:24]
)

In [ ]:
n.generators_t.r

In [ ]:
n.links_t.r.sum(axis=1)

In [ ]:
cap_constraints = n.generators_t.mu_ramp_upper.sum().add(
    n.generators_t.mu_capacity_upper.sum(),
    fill_value=0
).add(
    n.generators_t.mu_upper.sum(),
    fill_value=0
).add(
    n.generators_t.mu_ramp.sum(),
    fill_value=0
)


In [ ]:
cap_constraints = n.generators_t.mu_ramp_upper.sum().add(
    n.generators_t.mu_capacity_upper.sum(),
    fill_value=0
).add(
    n.generators_t.mu_upper.sum(),
    fill_value=0
).add(
    n.generators_t.mu_ramp.sum(),
    fill_value=0
)


In [ ]:
cap_constraints[cap_constraints!=0].plot.bar()

In [ ]:
contribution_margin = n.buses_t.marginal_price[n.generators.bus].subtract(n.generators.marginal_cost.values).clip(0)

In [ ]:
contribution_margin.columns = n.generators.index

In [ ]:
reserve_price = n.buses_t.reserve_price.reindex(n.generators.loc[n.generators_t.r.columns, "bus"].values,axis=1)

In [ ]:
reserve_price.columns = n.generators.index

In [ ]:
revenue_per_cap = n.generators_t.p.multiply(contribution_margin).sum().add(
    n.generators_t.r.multiply(
        reserve_price,
        fill_value=0
    ).sum()
).div(n.generators.p_nom)

In [ ]:
revenue_per_cap.sum()/cap_constraints.sum()

In [ ]:
(revenue_per_cap.sum() + cap_constraints.sum())/cap_constraints.sum()

In [ ]:
(revenue_per_cap.sum() + n.generators_t.mu_upper.sum().sum())/n.generators_t.mu_upper.sum().sum()

In [ ]:
n.generators_t.mu_upper.sum().sum()/revenue_per_cap.sum()

In [ ]:
n.generators_t.mu_upper.sum().loc[n.generators.bus.map(n.buses.carrier) == "electricity"].sort_values()

In [ ]:
n.generators_t.p["CY00 csp"].multiply(n.buses_t.marginal_price["CY00"]).sum()

In [ ]:
n.buses_t.marginal_price["GR03"].plot()

In [ ]:
revenue_per_cap.sort_values(ascending=False).loc["GR03 onwind"]

In [ ]:
n.generators_t.mu_upper.sum().sum()

In [ ]:
cap_constraints.sum()

In [ ]:
n.generators_t.mu_capacity_upper[unit][n.generators_t.mu_capacity_upper[unit]<-0.00000001]


In [ ]:
n.ge

In [ ]:
n.buses_t.reserve_price.dropna(how="all",axis=1).dropna().plot(legend=False)

In [ ]:
n = pypsa.Network("results/networks/base/cy1982_ty2025.nc")

In [ ]:
import datetime

In [ ]:
start = datetime.datetime.now()

n.generators.loc[n.generators.query("carrier in @dispatchable").index, "committable"] = False

n.optimize(solver_name="highs", snapshots=n.snapshots[:24])

end = datetime.datetime.now()

print(end-start)

In [ ]:
start = datetime.datetime.now()

n.generators.loc[n.generators.query("carrier in @dispatchable").index, "committable"] = True

n.optimize(solver_name="highs", snapshots=n.snapshots[:24], linearized_unit_commitment=True)

end = datetime.datetime.now()

print(end-start)

In [ ]:
snapshots = n.snapshots[:24]

In [ ]:
n.generators.loc[n.generators.committable == True, "min_down_time"] = 2
n.generators.loc[n.generators.committable == True, "min_up_time"] = 2

In [ ]:
m = n.optimize.create_model(linearized_unit_commitment=True, snapshots=snapshots)

In [ ]:
n.optimize(snapshots=snapshots, solver_name="highs", extra_functionality=reserves_unit_commitment)

In [ ]:
dirname = os.path.dirname(solved_network)

In [ ]:
if not os.path.isdir(dirname):
    os.makedirs(dirname)

In [ ]:
n.export_to_netcdf(solved_network)